In [2]:
import pandas as pd
import numpy as np
concrete_data = pd.read_csv('https://raw.githubusercontent.com/hsgeral/IBM-AI-Engineering-Professional-Certificate/main/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [3]:
concrete_data.shape

(1030, 9)

## Split data into predictors and target¶

In [4]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [5]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [6]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Normalize the data by substracting the mean and dividing by the standard deviation.

In [7]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()

In [8]:
n_cols = predictors_norm.shape[1] # number of predictors

## Start the Neural Network part

In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Defining a function that creates a NN with 
- One hidden layer of num_nodes, and a ReLU activation function
- Use the adam optimizer and the mean squared error  as the loss function.

In [11]:
# define regression model
def regression_model_1hidden(num_nodes, num_inputs, num_outputs):
    # create model
    model = Sequential()
    model.add(Dense(num_nodes, activation='relu', input_shape=(num_inputs,)))
    model.add(Dense(num_outputs))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Train and Test the Network
- using the fit method to train and test the model at the same time,
- leaving out 30% of the data for validation 
- using 50 epochs to train the model.
- repeating 50 times to collect the 50 mean squared errors.

The model will set apart the 'validation_split' fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. 

In [ ]:
num_nodes = 10
num_inputs = n_cols
num_outputs = 1
mse_list = [] # Initialize an empty list to store mean squared errors
for i in range(50):
    model = regression_model_1hidden(num_nodes, num_inputs, num_outputs) # build the model
    history=model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=0,shuffle=True) # fit the model
    mse_list.append(history.history['loss'][-1]) # get the mean square error after the 50th epoch
    print(i,mse_list[i])
# in each of the 50 repetitions the model was recreated otherwise the loss would constinously decrease as we were 
# fitting 50 * 50 epochs, increasing the number of epochs would result in much lower losses, but the exercised required to 
# limit it to 50
          
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)
print (mean_mse,std_mse)

0 372.6955171497784
1 244.6556513306172
2 462.9275278205184
3 299.4126801933892
4 517.487037436476
5 300.71973234009977
6 342.0605064952919
7 296.7337057296182
8 342.8161921825224
9 295.2041623013691
10 394.68808427829185
11 419.64041279489885
12 326.5332997146162
13 324.2822184357663
14 322.9371734280527
15 357.8505985508679
16 424.6111839178035
17 276.301227204512
18 289.296741374488
24 293.5405786860832
25 312.6164117355452
26 444.63656590814895
27 323.761428557884
28 365.05936350405796
29 344.2795748346887
30 481.23247266708563
31 297.9891853914512
32 737.6772098620622
33 294.85384746662623
34 297.2190893757856
35 331.8614743638799
36 487.7133994347179
37 398.6346261583982
38 333.3828331977749
39 473.81774699175406
40 328.97579651302175
